# PCA 분석

## 주성분 분석 개요

### 차원축소 (Dimensionlity Reduction)
데이터의 전반적인 특성을 보존하면서 데이터의 변수 수를 줄이는 방법

> 크기가 10행 7열인 dataframe의 차원은 (10x7) <br>
> 변수 3개를 제거하면 10행 4열로 바뀜 (10x4)

|방법|종류
|--|--|
|특성 선택|가장 중요한 특성들만 선택하여 기존의 데이터를 표현|
|특성 추출|기존 특성들을 사용하여 새로운 특성들을 만들어내는 방법|

### 주성분 분석(PCA)

데이터의 가장 큰 분산을 가진 방향으로 차원을 축소하여 데이터를 표현하는 방법

이를 통해 데이터를 가장 잘 설명하는 주요 특성들을 찾을 수 있다. 

데이터의 복잡성을 줄여 다양한 분야에서 활용되며 머신러닝, 패턴 인식, 시각화, 데이터 압축 등 다양한 분야에서 중요한 기술로 사용.


In [77]:
# 주성분 분석용 패키지
from sklearn.decomposition import PCA
# 주성분 분석 결과를 DataFrame으로 확인할 수 있는 기능을 제공하는 패키지
from pca import pca 
# 표준화 처리 패키지
from sklearn.preprocessing import StandardScaler

from pandas import read_excel, DataFrame

from matplotlib import pyplot as plt
import seaborn as sb

import sys
import os

import numpy as np
from pandas import DataFrame, MultiIndex, concat, DatetimeIndex, Series
from math import sqrt
from scipy.stats import t, pearsonr, spearmanr
from sklearn.impute import SimpleImputer
from scipy.stats import shapiro, normaltest, ks_2samp, bartlett, fligner, levene, chi2_contingency
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from pca import pca
from statsmodels.formula.api import logit
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, accuracy_score, recall_score, precision_score, f1_score

from matplotlib import pyplot as plt
import seaborn as sb
import sys

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from tabulate import tabulate


# sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
# from helper import my_ols

In [78]:
df=read_excel("C:\\Users\\MegaIT302-02\\Desktop\\한국복지패널_test.xlsx")
df.head()

,권역별지역구분,가처분소득,가구원번호,가구주와의관계,성별,태어난연도,교육수준,주된경제활동참여상태,자녀수,주택유형,...,농축산물돼지,농축산물닭,농축산물재고농산물,농축산물기타농축산물,자동차소유한비영업용자동차대수,자동차소유한자동차의가격,기타재산운동클럽등의회원권,기타재산운송및생계수단차량,기타재산귀금속골동품상품권등,기타재산기타
0,1,5958,1,10,1,1978,7,2,1,1,...,0,0,0,0,1,1500,0,0,90,0
1,1,5958,2,20,2,1975,7,2,1,1,...,0,0,0,0,1,1500,0,0,90,0
2,1,5958,3,11,2,2004,5,9,1,1,...,0,0,0,0,1,1500,0,0,90,0
3,1,5958,0,0,0,0,0,0,1,1,...,0,0,0,0,1,1500,0,0,90,0
4,2,1969,1,10,2,1961,4,6,1,1,...,0,0,0,0,0,0,0,0,0,0


### 데이터 전처리

#### 독립변수 컬럼만 추출

In [79]:
x_train = df.drop("교육비사교육비",axis=1)# y축에 해당하는 값을 삭제
x_train

,권역별지역구분,가처분소득,가구원번호,가구주와의관계,성별,태어난연도,교육수준,주된경제활동참여상태,자녀수,주택유형,...,농축산물돼지,농축산물닭,농축산물재고농산물,농축산물기타농축산물,자동차소유한비영업용자동차대수,자동차소유한자동차의가격,기타재산운동클럽등의회원권,기타재산운송및생계수단차량,기타재산귀금속골동품상품권등,기타재산기타
0,1,5958,1,10,1,1978,7,2,1,1,...,0,0,0,0,1,1500,0,0,90,0
1,1,5958,2,20,2,1975,7,2,1,1,...,0,0,0,0,1,1500,0,0,90,0
2,1,5958,3,11,2,2004,5,9,1,1,...,0,0,0,0,1,1500,0,0,90,0
3,1,5958,0,0,0,0,0,0,1,1,...,0,0,0,0,1,1500,0,0,90,0
4,2,1969,1,10,2,1961,4,6,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4786,5,8548,3,11,2,2006,4,9,2,6,...,0,0,0,0,2,7400,0,0,0,0
4787,5,8548,0,0,0,0,0,0,2,6,...,0,0,0,0,2,7400,0,0,0,0
4788,2,8452,1,10,1,1975,7,1,1,6,...,0,0,0,0,1,300,0,0,0,0
4789,2,8452,2,20,2,1976,7,9,1,6,...,0,0,0,0,1,300,0,0,0,0


### 추출된 독립변수를 표준화 

(D.탐색적 데이터 분석 -> 01_데이터정규화)

In [80]:
scaler = StandardScaler()
x_train_std = scaler.fit_transform(x_train)
x_train_std

array([[-1.36033388, -0.29344741, -0.50899384, ..., -0.19694104,
        -0.18560061, -0.0640251 ],
       [-1.36033388, -0.29344741,  0.44731576, ..., -0.19694104,
        -0.18560061, -0.0640251 ],
       [-1.36033388, -0.29344741,  1.40362536, ..., -0.19694104,
        -0.18560061, -0.0640251 ],
       ...,
       [-0.8684948 ,  0.35830852, -0.50899384, ..., -0.19694104,
        -0.43887336, -0.0640251 ],
       [-0.8684948 ,  0.35830852,  0.44731576, ..., -0.19694104,
        -0.43887336, -0.0640251 ],
       [-0.8684948 ,  0.35830852,  1.40362536, ..., -0.19694104,
        -0.43887336, -0.0640251 ]])

### 표준화 결과를 데이터 프레임으로 재구성

In [81]:
std_df = DataFrame(x_train_std, columns = x_train.columns)
std_df.head()

,권역별지역구분,가처분소득,가구원번호,가구주와의관계,성별,태어난연도,교육수준,주된경제활동참여상태,자녀수,주택유형,...,농축산물돼지,농축산물닭,농축산물재고농산물,농축산물기타농축산물,자동차소유한비영업용자동차대수,자동차소유한자동차의가격,기타재산운동클럽등의회원권,기타재산운송및생계수단차량,기타재산귀금속골동품상품권등,기타재산기타
0,-1.360334,-0.293447,-0.508994,-0.125504,-0.289155,0.485394,1.113279,-0.253741,-0.933851,-1.506911,...,0.0,-0.025815,-0.029779,-0.098862,-0.257446,-0.040223,-0.039707,-0.196941,-0.185601,-0.064025
1,-1.360334,-0.293447,0.447316,1.240134,1.046757,0.481580,1.113279,-0.253741,-0.933851,-1.506911,...,0.0,-0.025815,-0.029779,-0.098862,-0.257446,-0.040223,-0.039707,-0.196941,-0.185601,-0.064025
2,-1.360334,-0.293447,1.403625,0.011060,1.046757,0.518443,0.344787,1.743375,-0.933851,-1.506911,...,0.0,-0.025815,-0.029779,-0.098862,-0.257446,-0.040223,-0.039707,-0.196941,-0.185601,-0.064025
3,-1.360334,-0.293447,-1.465303,-1.491142,-1.625067,-2.028916,-1.576443,-0.824346,-0.933851,-1.506911,...,0.0,-0.025815,-0.029779,-0.098862,-0.257446,-0.040223,-0.039707,-0.196941,-0.185601,-0.064025
4,-0.868495,-1.335891,-0.508994,-0.125504,1.046757,0.463784,-0.039459,0.887468,-0.933851,-1.506911,...,0.0,-0.025815,-0.029779,-0.098862,-1.724063,-0.793043,-0.039707,-0.196941,-0.438873,-0.064025


In [82]:

    def ols(data, y, x):
        """
        회귀분석을 수행한다.

        Parameters
        -------
        - data : 데이터 프레임
        - y: 종속변수 이름
        - x: 독립변수의 이름들(리스트)
        """

        # 독립변수의 이름이 리스트가 아니라면 리스트로 변환
        if type(x) != list:
            x = [x]

        # 종속변수~독립변수1+독립변수2+독립변수3+... 형태의 식을 생성
        expr = "%s~%s" % (y, "+".join(x))

        # 회귀모델 생성
        model = ols(expr, data=data)
        # 분석 수행
        fit = model.fit()               

        # 파이썬 분석결과를 변수에 저장한다.
        summary = fit.summary()

        # 첫 번째, 세 번째 표의 내용을 딕셔너리로 분해
        my = {}

        for k in range(0, 3, 2):
            items = summary.tables[k].data
            # print(items)

            for item in items:
                # print(item)
                n = len(item)

                for i in range(0, n, 2):
                    key = item[i].strip()[:-1]
                    value = item[i+1].strip()

                    if key and value:
                        my[key] = value

        # 두 번째 표의 내용을 딕셔너리로 분해하여 my에 추가
        my['variables'] = []
        name_list = list(data.columns)
        # print(name_list)

        for i, v in enumerate(summary.tables[1].data):
            if i == 0:
                continue

            # 변수의 이름
            name = v[0].strip()

            vif = 0

            # Intercept는 제외
            if name in name_list:
                # 변수의 이름 목록에서 현재 변수가 몇 번째 항목인지 찾기
                j = name_list.index(name)
                vif = variance_inflation_factor(data, j)

            my['variables'].append({
                "name": name,
                "coef": v[1].strip(),
                "std err": v[2].strip(),
                "t": v[3].strip(),
                "P-value": v[4].strip(),
                "Beta": 0,
                "VIF": vif,
            })

        # 결과표를 데이터프레임으로 구성
        mylist = []
        yname_list = []
        xname_list = []

        for i in my['variables']:
            if i['name'] == 'Intercept':
                continue

            yname_list.append(y)
            xname_list.append(i['name'])

            item = {
                "B": i['coef'],
                "표준오차": i['std err'],
                "β": i['Beta'],
                "t": "%s*" % i['t'],
                "유의확률": i['P-value'],
                "VIF": i["VIF"]
            }

            mylist.append(item)

        table = DataFrame(mylist,
                          index=MultiIndex.from_arrays([yname_list, xname_list], names=['종속변수', '독립변수']))

        # 분석결과
        result = "𝑅(%s), 𝑅^2(%s), 𝐹(%s), 유의확률(%s), Durbin-Watson(%s)" % (
            my['R-squared'], my['Adj. R-squared'], my['F-statistic'], my['Prob (F-statistic)'], my['Durbin-Watson'])

        # 모형 적합도 보고
        goodness = "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p < 0.05)." % (y, ",".join(x), "유의하다" if float(
            my['Prob (F-statistic)']) < 0.05 else "유의하지 않다", my['Df Model'], my['Df Residuals'], my['F-statistic'])

        # 독립변수 보고
        varstr = []

        for i, v in enumerate(my['variables']):
            if i == 0:
                continue

            s = "%s의 회귀계수는 %s(p%s0.05)로, %s에 대하여 %s."
            k = s % (v['name'], v['coef'], "<" if float(v['P-value']) < 0.05 else '>', y,
                     '유의미한 예측변인인 것으로 나타났다' if float(v['P-value']) < 0.05 else '유의하지 않은 예측변인인 것으로 나타났다')

            varstr.append(k)

        ols_result = OlsResult()
        ols_result.model = model
        ols_result.fit = fit
        ols_result.summary = summary
        ols_result.table = table
        ols_result.result = result
        ols_result.goodness = goodness
        ols_result.varstr = varstr

        return ols_result


## PCA 패키지를 사용한 분석 (추천)

In [83]:
# 주성분 분석의 대상 컬럼 수를 독립변수의 전체 컬럼의 길이로 설정
model = pca(n_components=len(std_df.columns))
# 표준화 결과를 활용하여 주성분 분석 수행
fit = model.fit_transform(std_df)
fit

[pca] >Extracting column labels from dataframe.
[pca] >Extracting row labels from dataframe.
[pca] >The PCA reduction is performed on the [118] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Outlier detection using Hotelling T2 test with alpha=[0.05] and n_components=[118]
[pca] >Multiple test correction applied for Hotelling T2 test: [fdr_bh]
[pca] >Outlier detection using SPE/DmodX with n_std=[3]


{'loadings':             권역별지역구분         가처분소득         가구원번호       가구주와의관계            성별  \
 PC1   -5.185880e-02  2.180928e-01 -4.834021e-02 -1.219084e-01 -1.304508e-01   
 PC2   -2.054417e-02  1.058925e-01  1.095494e-01  2.403269e-01  2.207732e-01   
 PC3    7.446899e-02  6.828603e-02 -7.605556e-02 -7.600827e-02 -8.760493e-02   
 PC4    3.400834e-02 -5.181315e-03  1.335386e-02  5.966117e-02  3.245723e-02   
 PC5    7.072670e-02  7.192760e-02  2.549395e-02  2.470238e-02  5.184102e-03   
 ...             ...           ...           ...           ...           ...   
 PC114 -1.464728e-03 -1.720188e-03 -1.591221e-03  3.440529e-04 -6.791444e-04   
 PC115 -4.635248e-06  4.762226e-06  9.830944e-06 -1.371404e-07 -6.007526e-06   
 PC116 -1.393593e-17  2.100195e-16  5.380463e-16 -1.765728e-16  1.876006e-16   
 PC117 -0.000000e+00 -1.199225e-16 -2.145603e-16  1.378613e-16  4.339281e-17   
 PC118  0.000000e+00  1.350916e-16  1.108338e-16 -1.818558e-16 -1.067656e-16   
 
               태어난연도      

생성된 주성분에 사용된 필드 확인

In [84]:
topfit = fit['topfeat']
topfit

,PC,feature,loading,type
0,PC1,사회보장부담금,0.259435,best
1,PC2,나가족의수입만족도,0.304076,best
2,PC3,농어업정부보조금,0.255375,best
3,PC4,부모님교육수준모,-0.350987,best
4,PC5,아동수당,0.302465,best
...,...,...,...,...
148,PC54,농축산물소,0.274738,weak
149,PC44,농축산물닭,0.320757,weak
150,PC79,기타재산운동클럽등의회원권,0.334924,weak
151,PC25,기타재산운송및생계수단차량,-0.263286,weak


### 주성분 분석 결과를 토대로 회귀분석 수행

원본 데이터프레임으로 분석

In [85]:
best = topfit.query('type=="best"')
topfit = fit['topfeat']
topfit

,PC,feature,loading,type
0,PC1,사회보장부담금,0.259435,best
1,PC2,나가족의수입만족도,0.304076,best
2,PC3,농어업정부보조금,0.255375,best
3,PC4,부모님교육수준모,-0.350987,best
4,PC5,아동수당,0.302465,best
...,...,...,...,...
148,PC54,농축산물소,0.274738,weak
149,PC44,농축산물닭,0.320757,weak
150,PC79,기타재산운동클럽등의회원권,0.334924,weak
151,PC25,기타재산운송및생계수단차량,-0.263286,weak


In [86]:
best = topfit.query("type=='best'")
feature = list(set(best['feature']))
feature

['기타',
 '빚갚음',
 '태어난연도',
 '교양오락비',
 '농기계콤바인',
 '주택관련자금전세보증금포함',
 '세금소득세',
 '통신비',
 '부모님교육수준모',
 '장애수당장애인연금경증장애수당또는장애아동수당',
 '생활비생계비',
 '기타재산귀금속골동품상품권등',
 '사회보장부담건강보험료',
 '에너지감면또는보조전기료가스비등',
 '가구원번호',
 '지방보조금지역형생계긴급복지지원청년수당등',
 '금융자산기타',
 '총생활비',
 '권역별지역구분',
 '기타부채',
 '세금종합소득세',
 '성별',
 '식료품비주류담배비',
 '가구주와의관계',
 '재난지원금코로나19관련지원금',
 '외상미리탄곗돈',
 '사회보장부담고용보험료',
 '농기계경운기',
 '아전반적만족도',
 '근로장려세제',
 '마직업만족도',
 '광열수도비',
 '기타이자',
 '학비지원농림부지원포함',
 '주된경제활동참여상태',
 '가정위탁금또는소년소녀가장보호비',
 '보육료지원i사랑카드아이즐거운카드',
 '주거비월세',
 '사회보장부담금',
 '교육비공교육비',
 '보건의료비',
 '기초연금',
 '카드빚',
 '교육학자금마련포함',
 '금융기관대출',
 '금융자산계돈타기전',
 '비동거가구원에게보낸월평균송금보조비',
 '생계가곤란하여학업중단경험',
 '라가족관계만족도',
 '점유부동산기타권리금등',
 '경조금',
 '농축산물기타농축산물',
 '금융자산적금',
 '나가족의수입만족도',
 '식료품비가정식비',
 '가구가사용품비가구집기가사용품비',
 '교육수준',
 '최종학력',
 '가구가사용품비보육료비',
 '기타소비지출',
 '기타보조금',
 '기타바우처지원금',
 '농기계기타농기계',
 '세금',
 '농축산물재고농산물',
 '금융자산주식채권',
 '세금재산세',
 '아동수당',
 '교육비사교육비.1',
 '자동차소유한자동차의가격',
 '국가유공자에대한보조금보훈연금제외',
 '자동차소유한비영업용자동차대수',
 '소유부동산주택',
 '부모에게보낸월평균송금보조비',
 '피복신발비',
 '

In [87]:

def normalityTest(*any, isPrint=True):
    """
    분산분석을 수행하기 위한 정규성을 검정 한다.

    Parameters
    -------
    - any: 필드들

    Returns
    -------
    - df: 검정 결과 데이터 프레임
    """
    names = []

    result = {
        'field': [],
        'test': [],
        'statistic': [],
        'p-value': [],
        'result': []
    }
    for i in any:
        s, p = shapiro(i)
        result['field'].append(i.name)
        result['test'].append('shapiro')
        result['statistic'].append(s)
        result['p-value'].append(p)
        result['result'].append(p > 0.05)
        names.append('정규성')

    for i in any:
        s, p = normaltest(i)
        result['field'].append(i.name)
        result['test'].append('shapiro')
        result['statistic'].append(s)
        result['p-value'].append(p)
        result['result'].append(p > 0.05)
        names.append('정규성')

    n = len(any)

    for i in range(0, n):
        j = i + 1 if i < n - 1 else 0

        s, p = ks_2samp(any[i], any[j])
        result['field'].append(f'{any[i].name} vs {any[j].name}')
        result['test'].append('ks_2samp')
        result['statistic'].append(s)
        result['p-value'].append(p)
        result['result'].append(p > 0.05)
        names.append('정규성')

    rdf = DataFrame(result, index=names)

    if isPrint:
        prettyPrint(rdf)
    else:
        return rdf

def equalVarianceTest(*any, isPrint=True):
    """
    분산분석을 수행하기 위한 등분산성을 검정 한다.

    Parameters
    -------
    - any: 필드들

    Returns
    -------
    - df: 검정 결과 데이터 프레임
    """
    s1, p1 = bartlett(*any)
    s2, p2 = fligner(*any)
    s3, p3 = levene(*any)

    names = []

    for i in any:
        names.append(i.name)

    fix = " vs "
    name = fix.join(names)
    index = ['등분산성', '등분산성', '등분산성']

    df = DataFrame({
        'field': [name, name, name],
        'test': ['Bartlett', 'Fligner', 'Levene'],
        'statistic': [s1, s2, s3],
        'p-value': [p1, p2, p3],
        'result': [p1 > 0.05, p2 > 0.05, p3 > 0.05]
    }, index=index)

    if isPrint:
        prettyPrint(df)
    else:
        return df

def independenceTest(*any, isPrint=True):
    """
    분산분석을 수행하기 위한 독립성을 검정한다.

    Parameters
    -------
    - any: 필드들

    Returns
    -------
    - df: 검정 결과 데이터 프레임
    """
    df = DataFrame(any).T
    result = chi2_contingency(df)

    names = []

    for i in any:
        names.append(i.name)

    fix = " vs "
    name = fix.join(names)

    index = ['독립성']

    df = DataFrame({
        'field': [name],
        'test': ['Chi2'],
        'statistic': [result.statistic],
        'p-value': [result.pvalue],
        'result': [result.pvalue > 0.05]
    }, index=index)

    if isPrint:
        prettyPrint(df)
    else:
        return df









def allTest(*any, isPrint=True):
    """
    정규성, 등분산성, 독립성을 모두 검정한다.

    Parameters
    -------
    - any: 필드들

    Returns
    -------
    - df: 검정 결과 데이터 프레임
    """
    cc = concat([normalityTest(*any, isPrint=False), equalVarianceTest(*any, isPrint=False), independenceTest(*any, isPrint=False)])

    if isPrint:
        prettyPrint(cc)
    else:
        return cc

def pearson_r(df, isPrint=True):
    """
    피어슨 상관계수를 사용하여 상관분석을 수행한다.

    Parameters
    -------
    - df: 데이터 프레임

    Returns
    -------
    - rdf: 상관분석 결과 데이터 프레임
    """
    names = df.columns
    n = len(names)
    pv = 0.05

    data = []

    for i in range(0, n):
        # 기본적으로 i 다음 위치를 의미하지만 i가 마지막 인덱스일 경우 0으로 설정
        j = i + 1 if i < n - 1 else 0

        fields = names[i] + ' vs ' + names[j]
        s, p = pearsonr(df[names[i]], df[names[j]])
        result = p < pv

        data.append({'fields': fields, 'statistic': s,
                    'pvalue': p, 'result': result})

    rdf = DataFrame(data)
    rdf.set_index('fields', inplace=True)

    if isPrint:
        prettyPrint(rdf)
    else:
        return rdf

def spearman_r(df, isPrint=True):
    """
    스피어만 상관계수를 사용하여 상관분석을 수행한다.

    Parameters
    -------
    - df: 데이터 프레임

    Returns
    -------
    - rdf: 상관분석 결과 데이터 프레임
    """
    names = df.columns
    n = len(names)
    pv = 0.05

    data = []

    for i in range(0, n):
        # 기본적으로 i 다음 위치를 의미하지만 i가 마지막 인덱스일 경우 0으로 설정
        j = i + 1 if i < n - 1 else 0

        fields = names[i] + ' vs ' + names[j]
        s, p = spearmanr(df[names[i]], df[names[j]])
        result = p < pv

        data.append({'fields': fields, 'statistic': s,
                    'pvalue': p, 'result': result})

    rdf = DataFrame(data)
    rdf.set_index('fields', inplace=True)

    if isPrint:
        prettyPrint(rdf)
    else:
        return rdf

class OlsResult:
    def __init__(self):
        self._model = None
        self._fit = None
        self._summary = None
        self._table = None
        self._result = None
        self._goodness = None
        self._varstr = None

    @property
    def model(self):
        """
        분석모델
        """
        return self._model

    @model.setter
    def model(self, value):
        self._model = value

    @property
    def fit(self):
        """
        분석결과 객체
        """
        return self._fit

    @fit.setter
    def fit(self, value):
        self._fit = value

    @property
    def summary(self):
        """
        분석결과 요약 보고
        """
        return self._summary

    @summary.setter
    def summary(self, value):
        self._summary = value

    @property
    def table(self):
        """
        결과표
        """
        return self._table

    @table.setter
    def table(self, value):
        self._table = value

    @property
    def result(self):
        """
        결과표 부가 설명
        """
        return self._result

    @result.setter
    def result(self, value):
        self._result = value

    @property
    def goodness(self):
        """
        모형 적합도 보고
        """
        return self._goodness

    @goodness.setter
    def goodness(self, value):
        self._goodness = value

    @property
    def varstr(self):
        """
        독립변수 보고
        """
        return self._varstr

    @varstr.setter
    def varstr(self, value):
        self._varstr = value

def myOls(data, y=None, x=None, expr=None):
    """
    로지스틱 회귀분석을 수행한다.

    Parameters
    -------
    - data : 데이터 프레임
    - y: 종속변수 이름
    - x: 독립변수의 이름들(리스트)
    """

    # 데이터프레임 복사
    df = data.copy()

    # 종속변수~독립변수1+독립변수2+독립변수3+... 형태의 식을 생성
    if not expr:
        # 독립변수의 이름이 리스트가 아니라면 리스트로 변환
        if type(x) != list:
            x = [x]
        expr = "%s~%s" % (y, "+".join(x))
    else:
        x = []
        p = expr.find('~')
        y = expr[:p].strip()
        x_tmp = expr[p+1:]
        x_list = x_tmp.split('+')
            
        for i in x_list:
            k = i.strip()
                
            if k:
                x.append(k)

    # 회귀모델 생성
    model = ols(expr, data=data)
    # 분석 수행
    fit = model.fit()

    # 파이썬 분석결과를 변수에 저장한다.
    summary = fit.summary()

    # 첫 번째, 세 번째 표의 내용을 딕셔너리로 분해
    my = {}

    for k in range(0, 3, 2):
        items = summary.tables[k].data
        # print(items)

        for item in items:
            # print(item)
            n = len(item)

            for i in range(0, n, 2):
                key = item[i].strip()[:-1]
                value = item[i+1].strip()

                if key and value:
                    my[key] = value

    # 두 번째 표의 내용을 딕셔너리로 분해하여 my에 추가
    my['variables'] = []
    name_list = list(data.columns)
    #print(name_list)

    for i, v in enumerate(summary.tables[1].data):
        if i == 0:
            continue

        # 변수의 이름
        name = v[0].strip()

        vif = 0

        # Intercept는 제외
        if name in name_list:
            # 변수의 이름 목록에서 현재 변수가 몇 번째 항목인지 찾기 
            j = name_list.index(name)
            vif = variance_inflation_factor(data, j)

        my['variables'].append({
            "name": name,
            "coef": v[1].strip(),
            "std err": v[2].strip(),
            "t": v[3].strip(),
            "P-value": v[4].strip(),
            "Beta": 0,
            "VIF": vif,
        })

    # 결과표를 데이터프레임으로 구성
    mylist = []
    yname_list = []
    xname_list = []

    for i in my['variables']:
        if i['name'] == 'Intercept':
            continue

        yname_list.append(y)
        xname_list.append(i['name'])

        item = {
            "B": i['coef'],
            "표준오차": i['std err'],
            "β": i['Beta'],
            "t": "%s*" % i['t'],
            "유의확률": i['P-value'],
            "VIF": i["VIF"]
        }

        mylist.append(item)

    table = DataFrame(mylist,
                   index=MultiIndex.from_arrays([yname_list, xname_list], names=['종속변수', '독립변수']))
    
    # 분석결과
    result = "𝑅(%s), 𝑅^2(%s), 𝐹(%s), 유의확률(%s), Durbin-Watson(%s)" % (my['R-squared'], my['Adj. R-squared'], my['F-statistic'], my['Prob (F-statistic)'], my['Durbin-Watson'])

    # 모형 적합도 보고
    goodness = "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p < 0.05)." % (y, ",".join(x), "유의하다" if float(my['Prob (F-statistic)']) < 0.05 else "유의하지 않다", my['Df Model'], my['Df Residuals'], my['F-statistic'])

    # 독립변수 보고
    varstr = []

    for i, v in enumerate(my['variables']):
        if i == 0:
            continue
        
        s = "%s의 회귀계수는 %s(p%s0.05)로, %s에 대하여 %s."
        k = s % (v['name'], v['coef'], "<" if float(v['P-value']) < 0.05 else '>', y, '유의미한 예측변인인 것으로 나타났다' if float(v['P-value']) < 0.05 else '유의하지 않은 예측변인인 것으로 나타났다')

        varstr.append(k)

    ols_result = OlsResult()
    ols_result.model = model
    ols_result.fit = fit
    ols_result.summary = summary
    ols_result.table = table
    ols_result.result = result
    ols_result.goodness = goodness
    ols_result.varstr = varstr

    return ols_result

def scalling(df, yname=None):
    """
    데이터 프레임을 표준화 한다.

    Parameters
    -------
    - df: 데이터 프레임
    - yname: 종속변수 이름

    Returns
    -------
    - x_train_std_df: 표준화된 독립변수 데이터 프레임
    - y_train_std_df: 표준화된 종속변수 데이터 프레임
    """
    # 평소에는 yname을 제거한 항목을 사용
    # yname이 있지 않다면 df를 복사
    x_train = df.drop([yname], axis=1) if yname else df.copy()
    x_train_std = StandardScaler().fit_transform(x_train)
    x_train_std_df = DataFrame(x_train_std, columns=x_train.columns)
    
    if yname:
        y_train = df.filter([yname])
        y_train_std = StandardScaler().fit_transform(y_train)
        y_train_std_df = DataFrame(y_train_std, columns=y_train.columns)

    if yname:
        result = (x_train_std_df, y_train_std_df)
    else:
        result = x_train_std_df

    return result

def getBestFeatures(x_train_std_df):
    pca_model = pca()
    fit = pca_model.fit_transform(x_train_std_df)
    topfeat_df = fit['topfeat']
    
    best = topfeat_df.query("type=='best'")
    feature = list(set(list(best['feature'])))
    
    return (feature, topfeat_df)

class LogitResult:
    def __init__(self):
        self._model = None    
        self._fit = None
        self._summary = None
        self._prs = None
        self._cmdf = None
        self._result_df = None
        self._odds_rate_df = None

    @property
    def model(self):
        return self._model

    @model.setter
    def model(self, value):
        self._model = value

    @property
    def fit(self):
        return self._fit

    @fit.setter
    def fit(self, value):
        self._fit = value

    @property
    def summary(self):
        return self._summary

    @summary.setter
    def summary(self, value):
        self._summary = value

    @property
    def prs(self):
        return self._prs

    @prs.setter
    def prs(self, value):
        self._prs = value

    @property
    def cmdf(self):
        return self._cmdf

    @cmdf.setter
    def cmdf(self, value):
        self._cmdf = value

    @property
    def result_df(self):
        return self._result_df

    @result_df.setter
    def result_df(self, value):
        self._result_df = value

    @property
    def odds_rate_df(self):
        return self._odds_rate_df

    @odds_rate_df.setter
    def odds_rate_df(self, value):
        self._odds_rate_df = value

In [88]:
df.columns

Index(['권역별지역구분', '가처분소득', '가구원번호', '가구주와의관계', '성별', '태어난연도', '교육수준',
       '주된경제활동참여상태', '자녀수', '주택유형',
       ...
       '농축산물돼지', '농축산물닭', '농축산물재고농산물', '농축산물기타농축산물', '자동차소유한비영업용자동차대수',
       '자동차소유한자동차의가격', '기타재산운동클럽등의회원권', '기타재산운송및생계수단차량', '기타재산귀금속골동품상품권등',
       '기타재산기타'],
      dtype='object', length=119)

In [90]:
import numpy as np
import statsmodels.api as sm

# OLS 모델 생성 및 학습
model = sm.OLS('교육비사교육비', feature)
results = model.fit()

# 회귀 분석 결과 요약 출력
print(results.summary())

ValueError: unrecognized data structures: <class 'str'> / <class 'numpy.ndarray'>

In [ ]:
ols = myOls(df1,"교육비_사교육비",feature)
ols.summary

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
ols.table

B   표준오차  β        t   유의확률         VIF
종속변수 독립변수                                                   
MEDV RM         4.2847  0.420  0  10.206*  0.000  136.101743
     NOX      -15.8300  3.880  0  -4.079*  0.000   74.004269
     INDUS     -0.0722  0.061  0  -1.194*  0.233   14.485874
     TAX        0.0029  0.002  0   1.240*  0.216   61.939713
     DIS       -1.2542  0.187  0  -6.698*  0.000   15.430455
     CRIM      -0.0619  0.032  0  -1.905*  0.057    2.131404
     PTRATIO   -0.9655  0.124  0  -7.774*  0.000   87.227233
     CHAS       3.1229  0.880  0   3.548*  0.000    1.176266
     LSTAT     -0.5091  0.052  0  -9.786*  0.000   12.615188
     B          0.0086  0.003  0   3.112*  0.002   21.351015
     AGE       -0.0083  0.013  0  -0.616*  0.538   21.398863